In [10]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("schema match").getOrCreate()
data1=[(1,'A','A1',100)]
schema1=["id","name","dept","salary"]

data2=[(1,'A','A1',100)]
schema2=["id","name","dept","salary"]
data3=[(1,'A',1,100)]
schema3=["id","name","dept_id","salary"]

df1=spark.createDataFrame(data1,schema1)
df2=spark.createDataFrame(data2,schema2)
df3=spark.createDataFrame(data3,schema3)

df1.printSchema()
df2.printSchema()
df3.printSchema()

if(df1.schema == df2.schema):
    print("schema matched")
else: print("schema not matched")
    

if(df1.schema == df3.schema):
    print("schema matched")
else: print("schema not matched")

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- salary: long (nullable = true)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- salary: long (nullable = true)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- dept_id: long (nullable = true)
 |-- salary: long (nullable = true)

schema matched
schema not matched


In [18]:
# get the list of missing columns
print(list(set(df1.schema)-set(df3.schema)))
print(list(set(df3.schema)-set(df1.schema)))

[StructField('dept', StringType(), True)]
[StructField('dept_id', LongType(), True)]


In [20]:
# using columns: get missing columns list

df=df1.columns+df3.columns
uniqueCol=set(df)
print(df)
print(uniqueCol)

['id', 'name', 'dept', 'salary', 'id', 'name', 'dept_id', 'salary']
{'dept_id', 'salary', 'dept', 'name', 'id'}


In [21]:
# add missing column to df

from pyspark.sql.functions import lit
for i in uniqueCol:
    if i not in df1.columns:
        df1=df1.withColumn(i,lit(None))
    if i not in df3.columns:
        df3=df3.withColumn(i,lit(None))
        

display(df1)
display(df3)

DataFrame[id: bigint, name: string, dept: string, salary: bigint, dept_id: void]

DataFrame[id: bigint, name: string, dept_id: bigint, salary: bigint, dept: void]

In [22]:
df1.show()
df3.show()

+---+----+----+------+-------+
| id|name|dept|salary|dept_id|
+---+----+----+------+-------+
|  1|   A|  A1|   100|   null|
+---+----+----+------+-------+

+---+----+-------+------+----+
| id|name|dept_id|salary|dept|
+---+----+-------+------+----+
|  1|   A|      1|   100|null|
+---+----+-------+------+----+

